# NBA Sleepers

## By: Carlos Villagomez & Kevin Tran

## CPSC 310, Spring 2019

## Introduction:
We are using a dataset consisting of two decades worth of NBA player data ranging from the 1996-2016 seasons on each player's seasonal statistics who was a member of a team's roster. We implemented several classification tasks K-Nearest Neighbor (KNN), Naive Bayes, Linear Regression, and Sklearn Linear Regression. 

## Data Analysis:
The dataset consists of 21 attribute values including each player's demographic variables (e.g. height, weight, age, etc.), biographical details (team, draft year, round, etc.), and basic box score statistics (avg points, rebounds, assists, etc.). There are 9,561 total instances of players, but only 1,892 unique players in the dataset. Now, there were initially 51 rows of missing data, but the author of the dataset handled this by manually filling the gaps using player data from the online database, Basketball Reference.

We have used several models for performing classification. This includes K-Nearest Neighbor (KNN), Naive Bayes, Linear Regression, and Sklearn Linear Regression. It was found that KNN had the highest accuracy value when predicting a player's draft stock at 0.315, while the linear regression model only predicted a player's stock at an accuracy of 0.246.

## Classification Results:


## Conclusion:
The dataset we used did not present any inherent problems other than the 51 rows that were initially missing data, but were handled by the author filled in the missing data from Basketball Reference.